<a href="https://colab.research.google.com/github/dalsi137/nlpJ036/blob/master/ClassAssignment/NLP_24_Dec_Class_Assignment_6_J036.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import re
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

In [0]:
twitter = pd.read_csv('https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv', error_bad_lines=False)

In [3]:
twitter.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...


In [0]:
amazon = pd.read_json('http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz', lines=True)

In [5]:
twitter.columns

Index(['Topic', 'Sentiment', 'TweetId', 'TweetDate', 'TweetText'], dtype='object')

In [6]:
twitter['Sentiment'].unique()

array(['positive', 'negative', 'neutral', 'irrelevant'], dtype=object)

In [0]:
twitter = twitter[twitter['Sentiment']!='irrelevant']

In [0]:
def Preprocessing(text):
  #Initializing a dataframe of stopwords
  stop_words = pd.read_csv("https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords")
  stop_words.values.tolist()

  #Considering only alpha-numeric characters, numeric digits not even considered
  text = re.findall(r'\b[A-Za-z]\w+\b', text)
  
  #Remove stopwords
  for word in text:
    if word in stop_words:
      text.remove(word)
      
  #Remove numerics
  #res = ''.join([i for i in text if not i.isdigit()]) 

  #Remove @ and #
  bad_chars = ['@', '#']
  for i in text:
    if i in bad_chars: 
      text = text.remove(i)
  
  text = list(text)

  #Converting list to string
  text = ' '.join(word for word in text)
  return text

In [9]:

Preprocessing(twitter['TweetText'][0])


'Now all Apple has to do is get swype on the iphone and it will be crack Iphone that is'

In [0]:

twitter['preprocessed_text'] = twitter['TweetText'].apply(Preprocessing,1)

In [13]:
corpus = twitter['preprocessed_text'] 
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names()) #Column names
print(X.toarray()) #Array of observations

['a050tg4l', 'a0wnchmi', 'a11y', 'a1zgak5l', 'a6fllkge', 'a88zbr2o', 'a8on3iaa', 'a8tlwr3o', 'a8zqmpyp', 'aaa', 'aaargh', 'aac', 'aalkhubaizi', 'aap7stht', 'aapl', 'abandoned', 'abbas', 'abbreviations', 'ability', 'able', 'about', 'aboutthatlife', 'above', 'absolutely', 'abstract', 'abt', 'ac8jpjpk', 'acabo', 'academy', 'acap03y9', 'acappellamedia', 'acc', 'accelerate', 'accelerated', 'acceleration', 'accent', 'acceptable', 'access', 'accessibility', 'accessing', 'accessnetworks', 'accessories', 'accidently', 'accompanying', 'according', 'account', 'accounts', 'acct', 'accu', 'accu_med', 'accuracy', 'accused', 'accusing', 'acha', 'achieved', 'acjrl4fo', 'acquire', 'acquisition', 'acronyms', 'across', 'acting', 'action', 'activated', 'activating', 'active', 'activity', 'acts', 'actual', 'actualizar', 'actually', 'ad', 'adaddy101', 'adam', 'adamdince', 'adamnash', 'adcenter', 'add', 'added', 'addict', 'addicted', 'addicting', 'addictive', 'adding', 'additional', 'address', 'adds', 'addth

In [0]:
y = twitter['Sentiment']

In [0]:
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.25, random_state=42)

In [17]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [18]:
y_pred = rfc.predict(X_test)
y_pred[25:40]

array(['neutral', 'neutral', 'neutral', 'positive', 'positive', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral'], dtype=object)

In [19]:
confusion_matrix(y_test, y_pred)

array([[ 57,  97,   8],
       [ 17, 531,  14],
       [  7,  98,  27]])

In [20]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

    negative       0.35      0.70      0.47        81
     neutral       0.94      0.73      0.82       726
    positive       0.20      0.55      0.30        49

    accuracy                           0.72       856
   macro avg       0.50      0.66      0.53       856
weighted avg       0.85      0.72      0.76       856



In [0]:
def multiclass_roc_auc_score(y_test, y_pred, average='macro'):
  lb = LabelBinarizer()
  lb.fit(y_test)
  y_test = lb.transform(y_test)
  y_pred = lb.transform(y_pred)
  return roc_auc_score(y_test, y_pred, average=average)

In [22]:
multiclass_roc_auc_score(y_test, y_pred)

0.6288338308382949

In [0]:
le = LabelEncoder()
y_test = le.fit_transform(y_test)

In [24]:
pred_rf = rfc.predict_proba(X_test)
pred_rf

array([[0.2, 0.7, 0.1],
       [0.1, 0.8, 0.1],
       [0. , 0.9, 0.1],
       ...,
       [0.1, 0.8, 0.1],
       [0. , 0.6, 0.4],
       [0.4, 0.4, 0.2]])

In [0]:
def threshold6(array):
  a = []
  for i in range(len(array)):
    if max(array[i])>0.6:
      a.append(np.argmax(array[i]))
    else:
      a.append(2)
  return a

In [26]:
print("\n 0.6 threshold for Random Forest : \n",classification_report(threshold6(pred_rf),y_test))


 0.6 threshold for Random Forest : 
               precision    recall  f1-score   support

           0       0.12      0.91      0.22        22
           1       0.81      0.82      0.81       557
           2       0.59      0.28      0.38       277

    accuracy                           0.65       856
   macro avg       0.51      0.67      0.47       856
weighted avg       0.72      0.65      0.66       856



In [27]:
multiclass_roc_auc_score(threshold6(pred_rf),y_test)

0.7310279848968101

In [0]:
def threshold7(array):
  a = []
  for i in range(len(array)):
    if max(array[i])>0.7:
      a.append(np.argmax(array[i]))
    else:
      a.append(2)
  return a

In [29]:
print("\n 0.7 threshold for Random Forest : \n",classification_report((threshold7(pred_rf)),y_test))


 0.7 threshold for Random Forest : 
               precision    recall  f1-score   support

           0       0.07      0.92      0.14        13
           1       0.70      0.87      0.77       450
           2       0.79      0.26      0.40       393

    accuracy                           0.59       856
   macro avg       0.52      0.69      0.44       856
weighted avg       0.73      0.59      0.59       856



In [30]:
multiclass_roc_auc_score(threshold7(pred_rf),y_test)

0.7328339140275696

In [0]:
def threshold8(array):
  a = []
  for i in range(len(array)):
    if max(array[i])>0.8:
      a.append(np.argmax(array[i]))
    else:
      a.append(2)
  return a

In [32]:
print("\n 0.8 threshold for Random Forest : \n",classification_report((threshold8(pred_rf)),y_test))


 0.8 threshold for Random Forest : 
               precision    recall  f1-score   support

           0       0.01      1.00      0.02         2
           1       0.54      0.89      0.67       338
           2       0.87      0.22      0.35       516

    accuracy                           0.49       856
   macro avg       0.47      0.71      0.35       856
weighted avg       0.74      0.49      0.48       856



In [33]:
multiclass_roc_auc_score(threshold8(pred_rf),y_test)

0.729512534901256

In [35]:
amazon['preprocessed_text'] = amazon['reviewText'].apply(Preprocessing,1)

KeyboardInterrupt: ignored

In [36]:
corpus = amazon['preprocessed_text'] 
vectorizer = CountVectorizer()
x = vectorizer.fit_transform(corpus)

KeyError: ignored

In [0]:
amazon_pred = rfc.predict(X)
amazon_pred